In [1]:
import numpy as np
import cupy as cp
from pyqcu.cuda import define
from pyqcu.cuda import io
from pyqcu.cuda import qcu
from pyqcu.cuda.set import params, argv, set_ptrs
print('My rank is ', define.rank)
params[define._LAT_X_] = 4
params[define._LAT_Y_] = 4
params[define._LAT_Z_] = 4
params[define._LAT_T_] = 4
params[define._LAT_XYZT_] = params[define._LAT_X_] * \
    params[define._LAT_Y_]*params[define._LAT_Z_]*params[define._LAT_T_]
params[define._GRID_X_] = 1
params[define._GRID_Y_] = 1
params[define._GRID_Z_] = 1
params[define._GRID_T_] = 1
params[define._PARITY_] = 0
params[define._NODE_RANK_] = define.rank
params[define._NODE_SIZE_] = define.size
params[define._DATA_TYPE_] = define._LAT_C64_
dtype = define.dtype(_data_type_=params[define._DATA_TYPE_])
params[define._VERBOSE_] = 1
print("Parameters:", params)
argv[define._MASS_] = 0.0
print("Arguments:", argv)
#############################
gauge = cp.zeros(shape=[define._LAT_C_, define._LAT_C_, define._LAT_D_, params[define._LAT_T_],
                 params[define._LAT_Z_], params[define._LAT_Y_], params[define._LAT_X_]], dtype=dtype)
gauge = io.xxxtzyx2pxxxtzyx(input_array=gauge)
gauge = io.pccdtzyx2ccdptzyx(gauge=gauge)
print("Gauge Shape:", gauge.shape)


    @@@@@@######QCU NOTES START######@@@@@@@
    Guide:
    0. Required: MPI(e.g. 4.1.2), CUDA(e.g. 12.4), CMAKE(e.g. 3.22.1), GCC(e.g. 11.4.0), HDF5-MPI(e.g. 1.10.7,'apt install libhdf5-mpi-dev && export HDF5_MPI="ON" && pip install --no-binary=h5py h5py').
    1. The libqcu.so was compiled when pyqcu setup in download_path/PyQCU/lib, please add this path to your LD_LIBRARY_PATH.
    2. The QCU(PyQCU) splite grid by x->y->z->t, lattice by x->y->z->t->p->d[x,y,z,t]->c->c or x->y->z->t->c->s(->p) and x->y->z->t->c->s->c->s(->p).
    3. The QUDA(PyQUDA) splite grid by t->z->y->x, lattice by c->c->x->y->z->t->p->d[x,y,z,t] or c->s->x->y->z->t(->p) and c->s->c->s->x->y->z->t(->p).
    4. The QCU input params in numpy array(dtype=np.int32), argv in  numpy array(dtype=np.float32 or float64) array, set_ptrs in numpy array(dtype=np.int64), other in cupy array(dtype=cp.complex64 or complex128).
    5. The smallest lattice size is (wilson:x=4,y=4,z=4,t=4;clover:x=8,y=8,z=8,t=8) that QCU support

In [2]:
gauge.shape

(3, 3, 4, 2, 4, 4, 4, 2)

In [3]:

#############################
qcu.applyInitQcu(set_ptrs, params, argv)
qcu.applyGaussGaugeQcu(gauge, set_ptrs, params)
qcu.applyEndQcu(set_ptrs, params)
#############################



set_ptr:0x556c5f6eb7b0
long long set_ptr:93923945920432
gridDim.x               :8
blockDim.x              :16
host_params[_LAT_X_]    :2
host_params[_LAT_Y_]    :4
host_params[_LAT_Z_]    :4
host_params[_LAT_T_]    :4
host_params[_LAT_XYZT_] :128
host_params[_GRID_X_]   :1
host_params[_GRID_Y_]   :1
host_params[_GRID_Z_]   :1
host_params[_GRID_T_]   :1
host_params[_PARITY_]   :0
host_params[_NODE_RANK_]:0
host_params[_NODE_SIZE_]:1
host_params[_DAGGER_]   :0
host_params[_MAX_ITER_] :10000
host_params[_DATA_TYPE_]:3
host_params[_SET_INDEX_]:0
host_params[_SET_PLAN_] :0
host_params[_MG_X_]     :4
host_params[_MG_Y_]     :4
host_params[_MG_Z_]     :4
host_params[_MG_T_]     :8
host_params[_LAT_E_]    :24
host_params[_VERBOSE_]  :1
host_argv[_MASS_]       :0.000000e+00
host_argv[_TOL_]        :1.000000e-09
host_argv[_SIGMA_]      :1.000000e-01
lat_2dim[_XY_]          :8
lat_2dim[_XZ_]          :8
lat_2dim[_XT_]          :8
lat_2dim[_YZ_]          :16
lat_2dim[_YT_]          :16
lat_2dim[_

In [4]:
gauge.shape

(3, 3, 4, 2, 4, 4, 4, 2)

In [ ]:
_gauge=io.ccdptzyx2pccdtzyx(gauge=gauge)
_gauge = io.pxxxtzyx2xxxtzyx(input_array=_gauge)
index = 0
_U = _gauge[:, :, index, index, index, index, index]

Restored Array Shape: (3, 3, 4, 4, 4, 4, 4)


In [6]:
_U

array([[1.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j]], dtype=complex64)

In [7]:
_U.T

array([[1.+0.j, 0.+0.j, 0.+0.j],
       [1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j]], dtype=complex64)

In [8]:
_U.shape

(3, 3)

In [9]:
from pyqcu.cuda.gauge import test_su3
test_su3(_U)

U.conj().T @ U =
 [[1.+0.j 1.+0.j 0.+0.j]
 [1.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j]]
 Is SU(3) a member : False
U[6] = 0j, c6 = -0j
U[7] = 0j, c7 = -0j
U[8] = (1+0j), c8 = -0j
 Satisfy three-row complex conjugation properties : False


In [10]:
from cupyx.scipy.linalg import expm
A = cp.eye(3)
expm(a=A)

array([[2.71828183, 0.        , 0.        ],
       [0.        , 2.71828183, 0.        ],
       [0.        , 0.        , 2.71828183]])